In [1]:
from utils import *
import pandas as pd 
import numpy as np 
from tools.base import DATASETS_SYNTH, LRS

In [2]:
df_ks = pd.read_csv("../reports/schedule/KSWIN.csv.zip")
df_rest= pd.read_csv("../reports/schedule/v1.csv.zip")
df_ks

,base_lr,seed,batch_size,n_hidden_layers,n_hidden_units,gamma,factor,max_lr,schedule,optimizer,dataset,step,runtime,accuracy,lr,loss
0,2.000000,0,4,1,NaN,0.999756,1,1,Exponential Reset (KSWIN),SGD,Agrawal,100,0.021948,0.55,1.994152,0.803828
1,2.000000,0,4,1,NaN,0.999756,1,1,Exponential Reset (KSWIN),SGD,Agrawal,200,0.009720,0.65,1.982016,0.699952
2,2.000000,2,4,1,NaN,0.999756,1,1,Exponential Reset (KSWIN),SGD,Agrawal,100,0.021280,0.53,1.994152,0.792061
3,2.000000,0,4,1,NaN,0.999756,1,1,Exponential Reset (KSWIN),SGD,Agrawal,300,0.009619,0.66,1.969954,0.681246
4,2.000000,2,4,1,NaN,0.999756,1,1,Exponential Reset (KSWIN),SGD,Agrawal,200,0.009580,0.65,1.982016,0.738357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037595,0.003906,4,4,1,NaN,0.999939,1,1,Exponential Reset (KSWIN),SGD,Covertype,99600,0.010822,0.84,0.003906,0.144009
2037596,0.003906,4,4,1,NaN,0.999939,1,1,Exponential Reset (KSWIN),SGD,Covertype,99700,0.010972,0.88,0.003906,0.090634
2037597,0.003906,4,4,1,NaN,0.999939,1,1,Exponential Reset (KSWIN),SGD,Covertype,99800,0.010899,0.70,0.003906,0.152631
2037598,0.003906,4,4,1,NaN,0.999939,1,1,Exponential Reset (KSWIN),SGD,Covertype,99900,0.010984,0.86,0.003906,0.129376


In [3]:
pd.concat([df_ks, df_rest]).to_csv("../reports/schedule/v1.csv.zip", index=False)

In [4]:
def make_tables(df, datasets, ks, index, fname="schedules"):
    columns = [(dataset, "accuracy") for dataset in datasets]
    for k in ks:
        df = df[df["dataset"].isin(datasets)]
        table = get_topk_results(
            df,
            other_variables=["dataset", index],
            top_variable="base_lr",
            k=k,
        )
        table = table[["dataset", index, "accuracy_mean", "accuracy_std"]]
        table = col_to_header(table, "dataset", index=[index])
        mask_best = get_best_within_std(table, metrics=columns, modes="max")
        table = merge_mean_std(table, columns, formats="{:.2f}")
        style = table.style
        style = apply_style_attribute(style, mask=mask_best)
        style.to_latex(f"../pub/tables/{fname}_top{k}.tex", hrules=True)
    return style


# Define which data to show in table
datasets = [
    "Agrawal",
    "LED",
    "RBF abrupt",
    "RBF incr.",
    "Covertype",
    "Electricity",
    "Insects abrupt",
    "Insects gradual",
    "Insects incr.",
]

In [5]:
df = pd.read_csv("../reports/schedule/v1.csv.zip")
df["accuracy"] *= 100

In [6]:
filter_dicts = [
    {"schedule": "Step", "gamma": 0.75},
    {"schedule": "Step Reset", "gamma": 0.5},
    {"schedule": "Exponential", "gamma": 1 - 2**-13},
    {
        "schedule": "Exponential Reset",
        "gamma": 1 - 2**-12,
    },
    {
        "schedule": "Exponential Reset (KSWIN)",
        "gamma": 1 - 2**-12,
    },
    {
        "schedule": "Exponential Weight Reset",
        "gamma": 1 - 2**-12,
    },
    {"schedule": "Cyclic", "max_lr": 0.250},
    {"schedule": "Fixed"},
    {"schedule": "Kuncheva"},
]
df_table = filter_dataframe(df, filter_dicts)
df_table = df_table[["dataset", "schedule", "accuracy", "base_lr", "seed"]]

In [7]:
ks = [1, 3]
make_tables(df_table, datasets, ks, index="schedule")

,Agrawal,LED,RBF abrupt,RBF incr.,Covertype,Electricity,Insects abrupt,Insects gradual,Insects incr.
,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy
schedule,,,,,,,,,
Cyclic,78.07±.33,73.51±.14,87.28±3.59,57.60±1.34,83.40±.21,67.80±1.03,71.41±.20,75.14±.35,60.28±.08
Exponential,77.15±1.01,72.70±.25,86.15±2.78,50.54±.80,82.53±.19,72.73±.51,71.58±.06,75.04±.17,60.68±.18
Exponential Reset,77.45±.70,72.49±.10,86.75±2.36,51.93±.79,82.17±.22,72.25±.50,70.62±.18,74.60±.23,59.80±.11
Exponential Reset (KSWIN),77.56±1.14,72.49±.10,86.75±2.36,51.93±.79,82.17±.22,72.25±.50,70.62±.18,74.60±.23,59.80±.11
Exponential Weight Reset,71.80±.76,61.82±.83,67.25±1.20,59.04±.46,82.09±.22,70.23±.62,61.42±.20,67.62±.37,47.30±.24
Fixed,79.13±.30,73.54±.10,89.10±2.03,57.07±1.49,83.08±.18,73.12±.42,71.12±.08,74.50±.19,59.95±.06
Step,76.12±1.24,72.30±.21,84.16±3.67,49.48±.83,82.46±.20,72.81±.52,71.56±.16,74.97±.15,60.63±.13


In [18]:
df = pd.read_csv("../reports/optimizers/v1.csv.zip")
df['accuracy'] *= 100
df_table = df[["dataset", "optimizer", "accuracy", 'base_lr', 'seed']]

In [19]:
make_tables(df_table, datasets, ks, index="optimizer", fname='optimizers')

,Agrawal,LED,RBF abrupt,RBF incr.,Covertype,Electricity,Insects abrupt,Insects gradual,Insects incr.
,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy,accuracy
optimizer,,,,,,,,,
AdaGrad,76.23±1.58,71.21±.54,74.21±3.83,44.45±1.07,81.07±.22,76.64±1.92,74.31±.34,76.37±.56,61.67±1.27
Adam,78.50±2.71,73.52±.14,83.40±2.36,58.90±2.94,78.85±.22,69.23±.41,75.34±1.79,75.50±.68,63.89±.14
COCOB,78.21±1.12,73.88±.50,90.75±1.28,51.70±2.11,82.27±.46,84.30±.58,74.75±.11,77.57±.14,63.80±.12
DAdaptAdam,nan±nan,nan±nan,nan±nan,nan±nan,66.49±.35,54.06±.34,nan±nan,38.76±.06,14.93±.41
DAdaptLion,nan±nan,nan±nan,nan±nan,nan±nan,22.29±.02,57.42±.02,nan±nan,16.34±.05,16.67±.02
DAdaptSGD,60.16±.96,54.69±8.34,41.37±3.34,42.61±1.46,76.69±.79,66.03±1.75,50.05±11.26,48.21±10.62,36.00±11.81
DDoG,nan±nan,nan±nan,nan±nan,nan±nan,83.15±.31,67.02±.43,nan±nan,63.38±3.05,30.48±.53
DoG,78.77±2.25,73.34±.13,87.04±3.13,55.72±2.43,83.07±.64,71.53±.70,70.59±.26,74.01±.21,59.66±.22
